# 读取所有对象
Weaviate 提供必要的 API 来迭代所有数据。当您想要手动将数据（和向量嵌入）从一个地方复制/迁移到另一个地方时，这很有用。

after这是在运算符（也称为游标 API）的帮助下完成的。

迭代器
> 新的 API 客户端（目前由 Python Client v4 支持）将此功能封装为Iterator。

In [1]:
import json
import weaviate
from weaviate.auth import AuthApiKey

# 连接到本地部署的 Weaviate
client = weaviate.Client(
    url="http://127.0.0.1:8080",
    auth_client_secret=AuthApiKey("WVF5YThaHlkYwhGUSmCRgsX3tD5ngdN8pkih")
)

## 读取对象属性和 ID
以下代码遍历所有对象，提供每个对象的属性和 id。


In [6]:
# STEP 1 - Prepare a helper function to iterate through data in batches
def get_batch_with_cursor(collection_name, batch_size, cursor=None):
    # First prepare the query to run through data
    query = (
        client.query.get(
            collection_name,         # update with your collection name
            ["title"] # update with the required properties
        )
        .with_additional(["id"])
        .with_limit(batch_size)
    )

    # Fetch the next set of results
    if cursor is not None:
        result = query.with_after(cursor).do()
    # Fetch the first set of results
    else:
        result = query.do()

    print("----", result)

    return result["data"]["Get"][collection_name]

# STEP 2 - Iterate through the data
cursor = None
while True:
    # Get the next batch of objects
    next_batch = get_batch_with_cursor("MyCollection", 100, cursor)

    # Break the loop if empty – we are done
    if len(next_batch) == 0:
        break

    # Here is your next batch of objects
    print(next_batch)

    # Move the cursor to the last returned uuid
    cursor=next_batch[-1]["_additional"]["id"]

---- {'data': {'Get': {'MyCollection': [{'_additional': {'id': '250a842c-a36f-4ba1-a3b2-5c2e11bfc04d'}, 'title': 'Object 2'}, {'_additional': {'id': '261f7ad6-2517-443d-8238-db85919aaa25'}, 'title': 'Object 1'}, {'_additional': {'id': '42a6fedc-685c-441a-a84f-589da90710da'}, 'title': 'Object 4'}, {'_additional': {'id': '88a6f10c-1de8-4822-ae01-194853e0a886'}, 'title': 'Object 5'}, {'_additional': {'id': 'df75b23a-3e98-4ff1-85db-4bf8df9f8806'}, 'title': 'Object 3'}]}}}
[{'_additional': {'id': '250a842c-a36f-4ba1-a3b2-5c2e11bfc04d'}, 'title': 'Object 2'}, {'_additional': {'id': '261f7ad6-2517-443d-8238-db85919aaa25'}, 'title': 'Object 1'}, {'_additional': {'id': '42a6fedc-685c-441a-a84f-589da90710da'}, 'title': 'Object 4'}, {'_additional': {'id': '88a6f10c-1de8-4822-ae01-194853e0a886'}, 'title': 'Object 5'}, {'_additional': {'id': 'df75b23a-3e98-4ff1-85db-4bf8df9f8806'}, 'title': 'Object 3'}]
---- {'data': {'Get': {'MyCollection': []}}}


In [ ]:
### V4
collection = client.collections.get("WineReview")

for item in collection.iterator():
    print(item.uuid, item.properties)

## 读取所有对象，包括向量
读取包括向量在内的所有数据。（也适用于使用命名向量的情况。）

In [ ]:
# STEP 1 - Prepare a helper function to iterate through data in batches
def get_batch_with_cursor(collection_name, batch_size, cursor=None):
    # First prepare the query to run through data
    query = (
        client.query.get(
            collection_name,         # update with your collection name
            ["title", "description"] # update with the required properties
        )
        .with_additional(["id vector"])
        .with_limit(batch_size)
    )

    # Fetch the next set of results
    if cursor is not None:
        result = query.with_after(cursor).do()
    # Fetch the first set of results
    else:
        result = query.do()

    return result["data"]["Get"][collection_name]

# STEP 2 - Iterate through the data
cursor = None
while True:
    # Get the next batch of objects
    next_batch = get_batch_with_cursor("CollectionName", 100, cursor)

    # Break the loop if empty – we are done
    if len(next_batch) == 0:
        break

    # Here is your next batch of objects
    print(next_batch)

    # Move the cursor to the last returned uuid
    cursor=next_batch[-1]["_additional"]["id"]

In [ ]:
### V4

collection = client.collections.get("WineReview")

for item in collection.iterator(
    include_vector=True  # If using named vectors, you can specify ones to include e.g. ['title', 'body'], or True to include all
):
    print(item.properties)
    print(item.vector)

## 读取所有对象 - 多租户集合
遍历所有租户并读取每个租户的数据。

In [ ]:
# STEP 1 - Prepare a helper function to iterate through data in batches
def get_batch_with_cursor(collection_name, tenant_name, batch_size, cursor):
    # First prepare the query to run through data
    query = (
        client.query.get(
            collection_name,         # update with your collection name
            ["title", "description"] # update with the required properties
        )
        .with_tenant(tenant_name)     # tenant name goes here
        .with_additional(["id"])
        .with_limit(batch_size)
    )

    # Fetch the next set of results
    if cursor is not None:
        result = query.with_after(cursor).do()
    # Fetch the first set of results
    else:
        result = query.do()

    return result["data"]["Get"]["MultiTenancyCollection"]

# Get Tenants
tenants = client.schema.get_class_tenants(
    class_name="MultiTenancyCollection"  # The class from which the tenants will be retrieved
)

# STEP 2 - Iterate through Tenants
for tenant in tenants:
    # Reset the cursor to the beginning
    cursor = None
    while True:
        # Get the next batch of objects
        next_batch = get_batch_with_cursor("MultiTenancyCollection", tenant.name, 100, cursor)

        # Break the loop if empty – we are done
        if len(next_batch) == 0:
            break

        # Here is your next batch of objects
        print(next_batch)

        # Move the cursor to the last returned uuid
        cursor=next_batch[-1]["_additional"]["id"]

In [ ]:
multi_collection = client.collections.get("WineReviewMT")

# Get a list of tenants
tenants = multi_collection.tenants.get()

# Iterate through tenants
for tenant_name in tenants.keys():
    # Iterate through objects within each tenant
    for item in multi_collection.with_tenant(tenant_name).iterator():
        print(f"{tenant_name}: {item.properties}")